# 1

In [4]:
def fib(n):
  if n == 0:
    return "0"
  elif n == 1:
    return "01"
  
  prevprev = "0"
  prev = "01"
  newstr = "".join([prev, prevprev])
  for i in range(2, n):
    prevprev = prev
    prev = newstr
    newstr = "".join([prev, prevprev])
  
  return newstr

In [5]:
fib(1)

'01'

In [6]:
fib(3)

'01001'

In [7]:
for x in range(10):
  print(f"S{x} = {fib(x)}")

S0 = 0
S1 = 01
S2 = 010
S3 = 01001
S4 = 01001010
S5 = 0100101001001
S6 = 010010100100101001010
S7 = 0100101001001010010100100101001001
S8 = 0100101001001010010100100101001001010010100100101001010
S9 = 01001010010010100101001001010010010100101001001010010100100101001001010010100100101001001


# 2

In [9]:
def seq_count(seq, idx = 0, prevElem = None, count = 0, highestCount = 0):
  if idx == len(seq):
    # take the highest of the current count and the global highest
    return max(count, highestCount)
  elif idx == 0:
    # start the recursion
    return seq_count(seq, 1, seq[0], 1, 1)
  
  if prevElem == seq[idx]:
    # add a count
    return seq_count(seq, idx + 1, seq[idx], count + 1, highestCount)
  else:
    # start a new count
    highestCount = max(count, highestCount)
    return seq_count(seq, idx + 1, seq[idx], 1, highestCount)

In [10]:
seq_count([1, 3, 1, 1, 3, 3, 4, 4, 4])

3

In [11]:
seq_count((1, 3, 1, 1, 1, '1', 1, [3, 3, 3, 3], 3, 4, 0))

3

In [12]:
seq_count([[1], [1], [1], 1, 3, 3, 2, 3, 3, 3, 3, 2, 4, 0])

4

In [13]:
seq_count(('G', 'g', 'a', "a", '''a''', 2, 's', 's'))

3

In [14]:
seq_count([3, 1, int(True), 1, 1, 2, 3, 3])

4

In [15]:
seq_count((1, 3, None, 3, 3, 1, 3, 3, 4, 0))

2

In [23]:
# brute force
def pattern_count(seq, pattern, idx = 0, maxCount = 0):
  if idx == len(seq):
    return maxCount
  
  # do calculation starting from this idx
  i, j = idx, 0
  count = 0
  while i < len(seq) and seq[i] == pattern[j]:
    i += 1
    j = (j + 1) % len(pattern)
    if j == 0:
      # we have cycled through all elements in the pattern
      count += 1

  maxCount = max(count, maxCount)
  return pattern_count(seq, pattern, idx + 1, maxCount)

In [24]:
pattern_count([0, 1, 2, 1, 2, 3, 1, 2, 1, 2, 1, 2, 4, 1, 2], (1, 2))

3

In [25]:
pattern_count([], [2])

0

In [26]:
pattern_count(['ab', 'ab', 'a', 'a', 'b'], 'ab')

1

In [27]:
pattern_count('CGGACTACTAGACT', 'ACT')

2

In [28]:
pattern_count((1, (1, 1, 1, 1), 2, 1, 1, 1), [1, 1])

1

In [29]:
pattern_count(['ab', 'ab', 'a', 'a', 'b'], ('ab',))

2

# 3

In [31]:
from random import uniform
from scipy.special import binom
from numpy import sqrt, pi, exp, tan, cumsum
from scipy.stats import probplot
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ??